In [ ]:
!pip install fake-useragent pandas requests pyperclip

## Asks for token address

In [2]:
token_address = input("Enter the token address: ")

# Get the transfer data
This part will get all the transfers of a token and put it in a csv

In [3]:
from fake_useragent import UserAgent
import requests

In [ ]:
import os

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

# get creation date
url = f"https://api-v2.solscan.io/v2/token/transfer/export?address={token_address}&activity_type[]=ACTIVITY_SPL_MINT"
response = requests.get(url, headers={"User-Agent": get_random_user_agent()})
creation_date = int(response.text.split("\n")[1].split(",")[1])
print(f"Token creation date: {creation_date}")


start_date = creation_date
end_date = creation_date + 60 * 5

url = f"https://api-v2.solscan.io/v2/token/transfer/export?address={token_address}&block_time[]={start_date}&block_time[]={end_date}"
output_file = "transfer_data.csv"

headers = {
    "User-Agent": get_random_user_agent()
}
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # repeat request until response content is empty
    while response.content:
        mode = 'ab' if os.path.exists(output_file) else 'wb'
        with open(output_file, mode) as file:
            if mode == 'ab':
                # Skip the header row if appending
                file.write(response.content.split(b'\n', 1)[1])
            else:
                file.write(response.content)
        
        start_date += 60 * 5
        end_date += 60 * 5

        url = f"https://api-v2.solscan.io/v2/token/transfer/export?address={token_address}&block_time[]={start_date}&block_time[]={end_date}"
        print(f"Downloading transactions from {start_date} to {end_date}")
        response = requests.get(url, headers=headers)
        print(response.status_code)

    print(f"File downloaded successfully and saved as {output_file}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

403
<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;backgrou

IndexError: list index out of range

# Calculate the date range of the transactions

In [ ]:
import pandas as pd

In [51]:
# Load data into a DataFrame
df = pd.read_csv('transfer_data.csv')

# Extract min and max time values
start = df['Time'].min()
end = df['Time'].max()


print(f"Min Time: {start}")
print(f"Max Time: {end}")

Min Time: 1734722030
Max Time: 1734778597


# Get the price data using the date range
User must open the link and copy the hole raw data. If user copies the same thing twice it doesn't detect the copy

In [ ]:
import pyperclip
import time
import json

def generate_link():
    link = f"https://gmgn.ai/api/v1/token_kline/sol/{token_address}?resolution=1s&from={start}&to={end}"
    print("Generated Link:")
    print(link)


def wait_for_copy():
    last_clipboard_content = pyperclip.paste()  # Get the current clipboard content

    print("Waiting for a copy action...")
    while True:
        time.sleep(0.5)  # Poll every 0.5 seconds
        current_clipboard_content = pyperclip.paste()
        if current_clipboard_content != last_clipboard_content:
            print("Copy action detected!")
            return current_clipboard_content

generate_link()
json_data = wait_for_copy()
# segunda opcion
# json_data = input("Paste the copied JSON data here: ")

json_data = json.loads(json_data)

data_list = json_data['data']['list']

df = pd.DataFrame(data_list)
df.rename(columns={"time": "Time"}, inplace=True)

# Change the time column to be the first column
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

# Remove the volume column
df.drop(columns=['volume'], inplace=True)

# save the data into price_data.csv
df.to_csv('price_data.csv', index=False)

# Filter the transfers data (WIP)

In [ ]:
# Load the CSV files
csv1 = pd.read_csv("transfer_data.csv")
csv2 = pd.read_csv("transfer_data_to_delte.csv")

# Drop rows from csv1 that match any row in csv2
csv1_filtered = pd.merge(csv1, csv2, on=list(csv2.columns), how='left', indicator=True)
csv1_filtered = csv1_filtered[csv1_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])

csv1_filtered['Time'] = csv1_filtered['Time'] + 3600

# Save the filtered result to a new CSV file
csv1_filtered.to_csv("filtered_transfer_data.csv", index=False)

print("Filtered CSV saved as 'filtered_transfer_data.csv'")